In [33]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm

In [4]:
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000, )) 
Z = np.random.normal(1, 0, (1000,))
Y = X + Z + W + np.random.normal(0, 1, (1000,))

In [15]:
error = Y - X
np.corrcoef(X, error)[0, 1]

np.float64(0.5082241349019035)

In [16]:
error = Y - X - Z
np.corrcoef(X, error)[0, 1]

np.float64(0.5082241349019037)

In [21]:
hw7 = pd.read_csv('homework_7.1.csv', index_col=0)
hw7

,X,W,Z,Y
0,1.137055,1.221768,0.327829,1.944532
1,-0.112905,0.465835,0.599650,0.655514
2,2.077755,1.795414,-0.063393,5.934411
3,0.456373,-0.512159,1.177413,-0.188064
4,-1.012402,0.080002,-0.275697,-0.533775
...,...,...,...,...
9995,2.569934,1.233620,0.930467,6.188783
9996,0.190150,1.022164,-0.015151,0.697187
9997,-1.184465,-1.475929,-0.287056,-1.575303
9998,-0.121286,-0.914357,1.706237,-1.809819


In [36]:
def regress_subset(df, center, tol=0.1):
    subset = df[(df['W'] >= center - tol) & (df['W'] <= center + tol)]
    X_vars = subset[['X', 'Z']]
    X_vars = sm.add_constant(X_vars)
    model = sm.OLS(subset['Y'], X_vars).fit()
    return model.summary()

summary_W_neg1 = regress_subset(hw7, -1)
summary_W_0 = regress_subset(hw7, 0)
summary_W_1 = regress_subset(hw7, 1)
summary_W_neg1, summary_W_0, summary_W_1

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      Y   R-squared:                       0.510
 Model:                            OLS   Adj. R-squared:                  0.508
 Method:                 Least Squares   F-statistic:                     251.9
 Date:                Sun, 06 Jul 2025   Prob (F-statistic):           9.34e-76
 Time:                        14:27:58   Log-Likelihood:                -691.57
 No. Observations:                 488   AIC:                             1389.
 Df Residuals:                     485   BIC:                             1402.
 Df Model:                           2                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [37]:
def make_error(corr_const, num):
    err = list()
    prev = np.random.normal(0, 1)
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1)
        err.append(prev)
    return np.array(err)

In [40]:
n = 500
trials = 1000
corr_consts = [0.2, 0.5, 0.8]

results = {}

for rho in corr_consts:
    coefs = []
    std_errs = []

    for _ in range(trials):
        err_x = make_error(rho, n)
        err_y = make_error(rho, n)

        X = err_x
        Y = X + err_y
        X_design = sm.add_constant(X)
        model = sm.OLS(Y, X_design).fit()
        coefs.append(model.params[1])
        std_errs.append(model.bse[1])

    results[rho] = {
        'coef_std_dev': np.std(coefs),
        'stderr_mean': np.mean(std_errs)
    }

results

{0.2: {'coef_std_dev': np.float64(0.0476375534506768),
  'stderr_mean': np.float64(0.04474042144910821)},
 0.5: {'coef_std_dev': np.float64(0.05824787363560534),
  'stderr_mean': np.float64(0.04477015754694382)},
 0.8: {'coef_std_dev': np.float64(0.09779160954781542),
  'stderr_mean': np.float64(0.044608487024232764)}}

In [41]:
0.0476/0.0447

1.0648769574944073

In [42]:
0.0582/0.0447

1.302013422818792

In [43]:
0.0978/0.0447

2.1879194630872485

In [48]:
from scipy.spatial import distance

point1 = np.array([2, 1])
point2 = np.array([0, 0])

distance.minkowski(point1, point2, p=1.5)

np.float64(2.4472608147714756)

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

X = [[1, 2, 3]]
Y = [[2, 4, 6]]
cosine_similarity(X, Y)

array([[1.]])